In [ ]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 4.3 MB/s 
     |████████████████████████████████| 131 kB 35.8 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 7.7 MB 44.2 MB/s 
     |████████████████████████████████| 138 kB 48.7 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
import torch
import argparse
import numpy as np

In [ ]:
# 讀入測試文章
path = 'testing.txt'
f = open(path, 'r')
article = f.read()
f.close()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

def keywordextract(sentence, path):
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model = torch.load(path)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None, attention_mask=segments_tensors)
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

100%|██████████| 407873900/407873900 [00:29<00:00, 13637479.11B/s]


In [ ]:
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(description='BERT Keyword Extractor')
parser.add_argument('--sentence', type=str, default= article, help='sentence to get keywords')# default = target sentence
parser.add_argument('--path', type=str, default='model.pt', help='path to load model')
args = parser.parse_args()

In [ ]:
keywordextract(args.sentence, args.path)

language 0
language 0
